In [1]:
# Import libraries
import expectexception

# 4. Best Practices of Class Design

How do you design classes for inheritance? Does Python have private attributes? Is it possible to control attribute access? You'll find answers to these questions (and more) as you learn class design best practices.

# <font color=darkred>4.1 Designing for inheritance and polymorphism</font>

1. Designing for inheritance and polymorphism
>In this final chapter, we'll talk about some good practices of class design. We'll cover two main topics: efficient use of inheritance, and managing the levels of access to the data contained in your objects.

2. Polymorphism
>Polymorphism means using a unified interface to operate on objects of different classes. We've already dealt with it in Chapter 2.

3. Account classes
>We defined a bank account class, and two classes inherited from it: a checking account class and a savings account class. All of them had a withdraw method, but the checking account's method was executing different code.

4. All that matters is the interface
>Let's say we defined a function to withdraw the same amount of money from a whole list of accounts at once. This function doesn't know -- or care -- whether the objects passed to it are checking accounts, savings accounts or a mix -- all that matters is that they have a withdraw method that accepts one argument. That is enough to make the function work. It does not check which withdraw it should call -- the original or the modified. When the withdraw method is actually called, Python will dynamically pull the correct method: modified withdraw for whenever a checking account is being processed,and base withdraw for whenever a savings or generic bank account is processed. So you, as a person writing this batch processing function, don't need to worry about what exactly is being passed to it, only what kind of interface it has. To really make use of this idea, you have to design your classes with inheritance and polymorphism - the uniformity of interface - in mind

5. Liskov substitution principle
>There is a fundamental object-oriented design principle of when and how to use inheritance properly, called "Liskov substitution principle" named after the computer scientist Barbara Liskov: A base class should be interchangeable with any of its subclasses without altering any properties of the surrounding program. Using the example of our Account hierarchy, that means that wherever in your application you use a bankaccount object instance, substituting a checking account instead should not affect anything in the surrounding program. For example, the batch withdraw function worked regardless of what kind of account was used.

6. Liskov substitution principle
>This should be true both syntactically and semantically. On the one hand, the method in a subclass should have a signature with parameters and returned values compatible with the method in the parent class. On the other hand, the state of objects also must stay consistent; the subclass method shouldn't rely on stronger input conditions, should not provide weaker output conditions, it should not throw additional exceptions and so on.

7. Violating LSP
>Let's illustrate some possible violations of LSP - Liskov substitution principle - on our account classes: for example, the parent's -- or base's -- withdraw method could require 1 parameter, but the subclass method could require 2. Then we couldn't use the subclass's withdraw in place of the parent's. But if the subclass method has a default value for the second parameter, then there is no problem. If the subclass method only accepts certain amounts, unlike the base one, then sometimes the subclass could not be used in place of the base class, if those unsuitable amounts are used. If the base withdraw had a check for whether the resulting balance is positive, and only performed the withdrawal in that case, but the subclass did not do that, we wouldn't be able to use subclass in place of the base class, because it's possible that ambient program depends on the fact that the balance is always positive after withdrawal.

8. Violating LSP
>There are other ways to violate LSP like changing attributes that weren't changed in the base class, or throwing additional exceptions that the base class didn't throw (because those new exceptions are guaranteed to be unhandled).

9. No LSP - no inheritance
>The ultimate rule is that if your class hierarchy violates the Liskov substitution principle, then you should not be using inheritance, because it is likely to cause the code to behave in unpredictable ways somewhere down the road.

10. Let's practice!
>In the exercises, you'll explore the circle-ellipse problem - a famous software design problem that shows how our notions of inheritance are flawed. Have fun!

In [2]:
class BankAccount:
    def __init__(self, balance):
        self.balance = balance
    def withdraw(self, amount):
        self.balance -= amount 
    
class SavingsAccount(BankAccount):
    def __init__(self, balance, interest_rate=.1):
        # Call the parent constructor using ClassName.__init__()
        BankAccount.__init__(self, balance) # <--- self is a SavingsAccount but also a BankAccount
        # Add more functionality
        self.interest_rate = interest_rate
    def compute_interest(self, n_periods = 1):
        return self.balance * ( (1 + self.interest_rate) ** n_periods - 1)

class CheckingAccount(BankAccount):
    def __init__(self, balance, limit=200):
        BankAccount.__init__(self, balance)
        self.limit = limit
    def deposit(self, amount):
        self.balance += amount
    def withdraw(self, amount, fee=0):
        if fee <= self.limit:
            BankAccount.withdraw(self, amount - fee)
        else:
            BankAccount.withdraw(self, amount - self.limit)

########################################################################
## All that matters is the interface
########################################################################

# Withdraw amount from each of accounts in list_of_accounts
def batch_withdraw(list_of_accounts, amount=100):
    for acct in list_of_accounts:
        acct.withdraw(amount)
        print(acct.balance)

b, c, s = BankAccount(1000), CheckingAccount(2000), SavingsAccount(3000)
batch_withdraw([b,c,s]) # <-- Will use BankAccount.withdraw(),
                        # then CheckingAccount.withdraw(),
                        # then SavingsAccount.withdraw()

900
1900
2900


# <font color=darkred>4.2 Polymorphic methods</font>

**Instructions**

To design classes effectively, you need to understand how inheritance and polymorphism work together.

In this exercise, you have three classes - one parent and two children - each of which has a talk() method. Analyze the following code:

<code>
class Parent:
    def talk(self):
        print("Parent talking!")     
class Child(Parent):
    def talk(self):
        print("Child talking!")          
class TalkativeChild(Parent):
    def talk(self):
        print("TalkativeChild talking!")
        Parent.talk(self)
p, c, tc = Parent(), Child(), TalkativeChild()
for obj in (p, c, tc):
    obj.talk()
</code>

What is the output of the code above?

|1|2|3|4|
|-|-|-|-|
|Parent talking!<br>Parent talking!<br>Parent talking!|Parent talking!<br>Child talking!<br>Talkative Child talking!|Parent talking!<br>Child talking!<br>Parent talking!<br>Talkative Child talking!<br>Parent talking!|Parent talking!<br>Child talking!<br>Talkative Child talking!<br>Parent talking!|

You should be able to complete the exercise just by reading the code, without running it in the console!

**Possible Answers**
- 1
- 2
- 3
- <font color=red>4</font>
- Code causes an error

**Results**

<font color=darkgreen>Great job! Polymorphism ensures that the exact method called is determined dynamically based on the instance. What do you think would happen if Child did not implement talk()?</font>

In [3]:
class Parent:
    def talk(self):
        print("Parent talking!")     

class Child(Parent):
    def talk(self):
        print("Child talking!")          

class TalkativeChild(Parent):
    def talk(self):
        print("TalkativeChild talking!")
        Parent.talk(self)


p, c, tc = Parent(), Child(), TalkativeChild()

for obj in (p, c, tc):
    obj.talk()

Parent talking!
Child talking!
TalkativeChild talking!
Parent talking!


# <font color=darkred>4.3 Square and rectangle</font> 

The classic example of a problem that violates the Liskov Substitution Principle is the <a href='https://en.wikipedia.org/wiki/Circle%E2%80%93ellipse_problem'>Circle-Ellipse</a> problem, sometimes called the Square-Rectangle problem.

By all means, it seems like you should be able to define a class Rectangle, with attributes h and w (for height and width), and then define a class Square that inherits from the Rectangle. After all, a square "is-a" rectangle!

Unfortunately, this intuition doesn't apply to object-oriented design.

**Instructions**
- Create a class Rectangle with a constructor that accepts two parameters, h and w, and sets its h and w attributes to the values of h and w.
- Create a class Square inherited from Rectangle with a constructor that accepts one parameter w, and sets both the h and w attributes to the value of w.


- The classes are defined for you. Experiment with them in the console. For example, in the console or the script pane, create a Square object with side length 4. Then try assigning 7 to the h attribute. What went wrong with these classes?

    **Possible Answers**

    - This wasn't a correct use of inheritance: we did not call the parent constructor in the child constructor.
    - We cannot set the h attribute to 7 in the Square object because it will cause an error.
    - <font color=red>The 4x4 Square object would no longer be a square if we assign 7 to h.</font>
    - Because a Square only has one side length, it should not have the h attribute. We should not have included the h attribute in the constructor.<font color=gray>With inheritance, you can't pick and choose data to inherit. The child class (Square) has all the same data as the parent class (Rectangle).</font>
    - All of the above.


- A Square inherited from a Rectangle will always have both the h and w attributes, but we can't allow them to change independently of each other.
    - Define methods set_h() and set_w() in Rectangle, each accepting one parameter and setting h and w.
    - Define methods set_h() and set_w() in Square, each accepting one parameter, and setting both h and w to that parameter in both methods.
    

- Later in this chapter you'll learn how to make these setter methods run automatically when attributes are assigned new values, don't worry about that for now, just assume that when we assign a value to h of a square, now the w attribute will be changed accordingly. How does using these setter methods violate Liskov Substitution principle?

    **Possible Answers**
    - There are syntactic inconsistencies.
    - <font color=red>Each of the setter methods of Square change both h and w attributes, while setter methods of Rectangle change only one attribute at a time, so the Square objects cannot be substituted for Rectangle into programs that rely on one attribute staying constant.</font>
    - The setter methods of Square accept only limited range of parameters, unlike the setter methods of Rectangle, so the Square objects cannot be substituted for Rectangle into programs that use parameter values outside that range. <font color=gray>Good reasoning, but no: the parameter that the setter methods accept can be anything, both for the square and the rectangle. What's restricted is how the attributes change according to this parameter.</font>
    - All of the above.

**Results**

<font color=darkgreen>Correct! Remember that the substitution principle requires the substitution to preserve the oversall state of the program. An example of a program that would fail when this substitution is made is a unit test for a setter functions in Rectangle class.</font>

In [4]:
# Define a Rectangle class
class Rectangle:
    """Create a rectangle class with h and w"""
    def __init__(self, h, w):
        self.h, self.w = h, w

# Define a Square class
class Square(Rectangle):
    """Create a square class with w"""
    def __init__(self, w):
        self.h, self.w = w, w

s = Square(4)
s.h = 7

In [5]:
class Rectangle:
    def __init__(self, w,h):
      self.w, self.h = w,h
    
    # Define set_h to set h      
    def set_h(self, h):
      self.h = h
    
    # Define set_w to set w          
    def set_w(self, w):
      self.w = w
      
      
class Square(Rectangle):
    def __init__(self, w):
      self.w, self.h = w, w 
    
    # Define set_h to set w and h
    def set_h(self, h):
      self.h = h
      self.w = h
    
    # Define set_w to set w and h      
    def set_w(self, w):
      self.h = w
      self.w = w

# <font color=darkred>4.4 Managing data access: private attributes</font>

1. Managing data access: private attributes
>In the next two lessons, we'll talk about managing data access.

2. All class data is public
>All class data in Python is technically public. Any attribute or method of any class can be accessed by anyone. If you are coming from a background in another programming language like Java, this might seem unusual or an oversight, but it is by design. The fundamental principle behind much of Python design "we are all adults here". It is a philosophy that goes beyond just code, and describes how the Python community interacts with each other: you should have trust in your fellow developers.

3. Restricting access
>That said, there are a few ways to manage access to data. We can use some universal naming conventions to signal that the data is not for external consumption; then, there are special kinds of attributes called properties that allow you to control how each attribute is modified. Finally, there are special methods that you can override to change how attributes are used entirely. We'll cover the first two options in this chapter, and you are unlikely to ever need anything beyond that.

4. Naming convention: internal attributes
>Let's start with naming conventions. The first and most important convention is using a single leading underscore to indicate an attribute or method that isn't a part of the public class interface, and can change without notice. This convention is widely accepted among Python developers, so you should follow it both as a class developer and as a class user. Nothing is technically preventing you from using these attributes, but a single leading underscore is the developer's way of saying that you shouldn't. The class developer trusts that you are an adult and will be able to use the class responsibly. This convention is used for internal implementation details and helper functions. For example, a pandas DataFrame has an underscore-is_mixed_type attribute that indicates whether the DataFrame contains data of mixed types, and the datetime module contains a _ymd2ord function that converts a date into a number containing how many days have passed since January 1st of year 1.

5. Naming convention: pseudoprivate attributes
>Another naming convention is using a leading double underscore. Attributes and methods whose names start with a double underscore are the closest thing Python has to "private" fields and methods of other programming languages. In this case, it means that this data is not inherited - at least, not in a way you're used to, because Python implements name mangling: any name starting with a double underscore will be automatically prepended by the name of the class when interpreted by Python, and that new name will be the actual internal name of the attribute or method. The main use of these pseudo-private attributes is to prevent name clashes in child classes: you can't control what attributes or methods someone will introduce when inheriting from your class, and it's possible that someone will unknowingly introduce a name that already exists in you class, thus overriding the parent method or attribute! You can use double leading underscores to protect important attributes and methods that should not be overridden. Finally, be careful: leading AND trailing double underscores are only used for build-in Python methods like init, so your name should only start -- but not end! -- with double underscores.

6. Let's practice!
>Head over to the exercises to review these conventions. In the next video, you'll learn how to add a little more control to how attributes are used.

# <font color=darkred>4.5 Attribute naming conventions</font> 

In Python, all data is public. Instead of access modifiers common in languages like Java, Python uses naming conventions to communicate the developer's intention to class users, shifting the responsibility of safe class use onto the class user.

Python uses underscores extensively to signal the purpose of methods and attributes. In this exercise, you will match a use case with the appropriate naming convention.

**Instructions**
- Drag the cards into the bucket representing the most appropriate naming convention for the use case.

| \_name | \_\_name | \_\_name\_\_ |
|:-|:-|:-|
|A helper method that checks validity of an attribute's value but is not considered a part of class's public interface.|A "version" attribute that stores the current version of the class and should not be passed to child classess, who will have their own versions.|A method that is run whenever the object is printed.|

**Results**

<font color=darkgreen>Great job! The single leading underscore is a convention for internal details of implementation. Double leading underscores are used for attributes that should not be inherited to aviod name clashes in child classes. Finally, leading and trailing double underscores are reserved for built-in methods.</font>

# <font color=darkred>4.6 Using internal attributes</font> 

In this exercise, you'll return to the BetterDate class of Chapter 2. Your date class is better because it will use the sensible convention of having exactly 30 days in each month.

You decide to add a method that checks the validity of the date, but you don't want to make it a part of BetterDate's public interface.

The class BetterDate is available in the script pane.

**Instructions**
- Add a class attribute _MAX_DAYS storing the maximal number of days in a month - 30.
- Add another class attribute storing the maximal number of months in a year - 12. Use the appropriate naming convention to indicate that this is an internal attribute.
- Add an _is_valid() method that returns True if the day and month attributes are less than or equal to the corresponding maximum values, and False otherwise. Make sure to refer to the class attributes by their names!

**Results**

<font color=darkgreen>Great job! Notice that you were still able to use the _is_valid() method as usual. The single underscore naming convention is purely a convention, and Python doesn't do anything special with such attributes and methods behind the scenes. That convention is widely followed, though, so if you see an attribute name with one leading underscore in someone's class - don't use it! The class developer trusts you with this responsibility.</font>

In [6]:
# Add class attributes for max number of days and months
class BetterDate:
    _MAX_DAYS = 30
    _MAX_MONTHS = 12
    
    def __init__(self, year, month, day):
        self.year, self.month, self.day = year, month, day
        
    @classmethod
    def from_str(cls, datestr):
        year, month, day = map(int, datestr.split("-"))
        return cls(year, month, day)
    
    # Add _is_valid() checking day and month values
    def _is_valid(self):
        return (self.month<=BetterDate._MAX_MONTHS) & (self.day<=BetterDate._MAX_DAYS)
    
bd1 = BetterDate(2020, 4, 30)
print(bd1._is_valid())

bd2 = BetterDate(2020, 6, 45)
print(bd2._is_valid())

True
False


# <font color=darkred>4.7 Properties</font>

1. Properties
>Welcome back! In this last video, you'll learn about properties, which are a special kind of attribute that allow customized access.

2. Changing attribute values
>In the beginning of Chapter 1, you worked with an Employee class where you defined methods like set_salary that were used to set the values for attributes. Later, you learned about using the constructor to initialize the attributes . You also learned that you can access and change the attributes directly by assignment. But this means that with a simple equality we can assign anything to salary: a million, a negative number, or even the word "Hello". Salary is an important attribute, and that should not be allowed.

3. Changing attribute values
>So how do we control attribute access, validate it or even make the attribute read-only? We could modify the set_salary method, but that wouldn't help, because we could still use the dot syntax and assignment via equality.

4. Restricted and read-only attributes
>There is a precedent for such attribute management with classes that you already know. For example, if you have a pandas DataFrame with two columns, you can set the columns attribute to a list of 2 strings -- new names for the columns. But if you try to set the attribute to a list of different length, you'd get an error. Or, consider the shape attribute -- it cannot be changed at all!

5. @property
>We can implement similar behavior using the property decorator. Start by defining an "internal" attribute that will store the data. As we learned in the previous video, it is recommended to start the name with one leading underscore. Here, we defined a underscore-salary attribute. Next, we define a method whose name is the exact name we'd like the restricted attribute to have, and put a decorator "property" on it. In our case that method is called salary, without underscore, because that's how we'd like to refer to it. If we were writing a DataFrame class, this could be "columns", or "shape". The method just returns the actual internal attribute that is storing the data. To customize how the attribute is set, we implement a method with a decorator "attribute name"-dot-setter: salary-dot-setter in our case. The method itself is again named exactly the same as the property -- salary - and it will be called when a value is assigned to the property attribute. It has a self argument, and an argument that represents the value to be assigned. Here we raise an exception if the value is negative, otherwise change the internal attribute. So there are two methods called salary -- the name of the property -- that have different decorators. The method with property decorator returns the data, and the method with salary dot setter decorator implements validation and sets the attribute.

6. @property
>How does this work in practice? We can use this property just as if it was a regular attribute (remember the only real attribute we have is the internal underscore-salary). Use the dot syntax and equality sign to assign a value to the salary property. Then, the setter method will be called. If we try to assign a negative value to salary, an exception will be raised.

7. Why use @property?
>Properties are useful because the user of your class will not have to do anything special. They won't even be able to distinguish between properties and regular attributes. You, on the other hand, now have some control over the access.

8. Other possibilities
>There are a few other things you can do with properties: **if you do not define a setter method, the property will be read-only, like Dataframe shape**. A method with an attribute-name-dot-getter decorator will be called when the property's value is just retrieved, and the method with the attribute-name-dot-deleter -- when an attribute is deleted.

9. Let's practice!
>Alright! Time for you to create some properties by yourself.

## WITHOUT THE SETTER METHOD

In [7]:
from datetime import datetime

# Define SalaryError inherited from ValueError
class SalaryError(ValueError): pass

# Define BonusError inherited from SalaryError
class BonusError(SalaryError): pass

# Define Employee class
class Employee:
    """Create Employee objects."""
    MIN_SALARY = 30000
    MAX_BONUS = 5000
    
    def __init__(self, name, salary=0):
        self.name = name
        
        # If salary is too low
        if salary<Employee.MIN_SALARY:
            # Raise a SalaryError exception
            raise SalaryError("Salary is too low!")
        
        # Assign salary
        self.salary = salary
         
        self.hire_date = datetime.today()
    
    @property
    def salary(self):
        return self._salary
    
    def give_bonus(self, amount):
        if amount > Employee.MAX_BONUS:
           raise BonusError("The bonus amount is too high!")  
        if self.salary + amount <  Employee.MIN_SALARY:
           raise SalaryError("The salary after bonus is too low!")
        self.salary += amount

    def monthly_salary(self):
        return self.salary/12
    
    @classmethod
    def from_file(cls, filename):
        with open(filename, "r") as f:
            name = f.readline()
        return cls(name)
    
    def __str__(self):
        return "Employee name: {name}\nEmployee salary: ${salary:,.2f}".format(name=self.name, salary=self.salary)
    
    def __repr__(self):
        return 'Employee("{}", {})'.format(self.name, self.salary) 

In [8]:
%%expect_exception AttributeError
emp = Employee("Miriam Azari", 35000)
emp.salary             # <-- accessing the "property"

emp.salary = 60000
#emp.salary = -1000     # <-- @salary.setter 

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-8-af0a8bab065d> in <module>
----> 1 emp = Employee("Miriam Azari", 35000)
      2 emp.salary             # <-- accessing the "property"
      3 
      4 emp.salary = 60000
      5 #emp.salary = -1000     # <-- @salary.setter

<ipython-input-7-12e4105cb810> in __init__(self, name, salary)
     22 
     23         # Assign salary
---> 24         self.salary = salary
     25 
     26         self.hire_date = datetime.today()

AttributeError: can't set attribute


## WITH THE SETTER METHOD

In [9]:
from datetime import datetime

# Define SalaryError inherited from ValueError
class SalaryError(ValueError): pass

# Define BonusError inherited from SalaryError
class BonusError(SalaryError): pass

# Define Employee class
class Employee:
    """Create Employee objects."""
    MIN_SALARY = 30000
    MAX_BONUS = 5000
    
    def __init__(self, name, salary=0):
        self.name = name
        self._salary = salary
        self.hire_date = datetime.today()
    
    @property
    def salary(self):
        return self._salary
    
    @salary.setter
    def salary(self, new_salary):
        if new_salary < 0:
            raise SalaryError("Salary is too low!")
        self._salary = new_salary
    
    def give_bonus(self, amount):
        if amount > Employee.MAX_BONUS:
           raise BonusError("The bonus amount is too high!")  
        if self.salary + amount <  Employee.MIN_SALARY:
           raise SalaryError("The salary after bonus is too low!")
        self.salary += amount

    def monthly_salary(self):
        return self.salary/12
    
    @classmethod
    def from_file(cls, filename):
        with open(filename, "r") as f:
            name = f.readline()
        return cls(name)
    
    def __str__(self):
        return "Employee name: {name}\nEmployee salary: ${salary:,.2f}".format(name=self.name, salary=self.salary)
    
    def __repr__(self):
        return 'Employee("{}", {})'.format(self.name, self.salary) 

In [10]:
%%expect_exception SalaryError
emp = Employee("Miriam Azari", 35000)
emp.salary             # <-- accessing the "property"

emp.salary = 60000
emp.salary = -1000     # <-- @salary.setter 

---------------------------------------------------------------------------
SalaryError                               Traceback (most recent call last)
<ipython-input-10-f28d277a31ed> in <module>
      3 
      4 emp.salary = 60000
----> 5 emp.salary = -1000     # <-- @salary.setter

<ipython-input-9-22ab45fe01a5> in salary(self, new_salary)
     25     def salary(self, new_salary):
     26         if new_salary < 0:
---> 27             raise SalaryError("Salary is too low!")
     28         self._salary = new_salary
     29 

SalaryError: Salary is too low!


# <font color=darkred>4.8 What do properties do?</font>

**Instructions**

You could think of properties as attributes with built-in access control. They are especially useful when there is some additional code you'd like to execute when assigning values to attributes.

Which of the following statements is NOT TRUE about properties?

**Possible Answers**
- Properties can be used to implement "read-only" attributes
- Properties can prevent creation of new attributes via assignment
- Properties can be accessed using the dot syntax just like regular attributes
- Properties allow for validation of values that are assigned to them

**Results**

<font color=darkgreen>This statement is indeed not true! Properties control only one specific attribute that they're attached to. There are ways to prevent creating new attributes, but doing so would go against the "we're all adults here" principle.</font>

# <font color=darkred>4.9 Create and set properties</font> 

There are two parts to defining a property:

- first, define an "internal" attribute that will contain the data;
- then, define a @property-decorated method whose name is the property name, and that returns the internal attribute storing the data.


If you'd also like to define a custom setter method, there's an additional step:


- define another method whose name is exactly the property name (again), and decorate it with @prop_name.setter where prop_name is the name of the property. The method should take two arguments -- self (as always), and the value that's being assigned to the property.


In this exercise, you'll create a balance property for a Customer class - a better, more controlled version of the balance attribute that you worked with before.

**Instructions**
- Create a Customer class with the __init__() method that:
    - takes parameters name and new_bal,
    - assigns name to the attribute name,
    - raises a ValueError if new_bal is negative,
    - otherwise, assigns new_bal to the attribute _balance (with _).


- Add a method balance() with a @property decorator that returns the _balance attribute.


- Define another balance() method to serve as a setter, with the appropriate decorator and an additional parameter:
    - Raise a ValueError if the parameter is negative,
    - otherwise assign it to _balance ;
    - print "Setter method is called".


- Create a Customer named Belinda Lutz with the balance of 2000 and save it as cust.
- Use the dot syntax and the = to assign 3000 to cust.balance.
- Print cust.balance.
- In the console, try assigning -1000 to cust.balance. What happens?


**Results**

<font color=darkgreen>Great start! Now the user of your Customer class won't be able to assign arbitrary values to the customers' balance. You could also add a custom getter method (with a decorator @balance.getter) that returns a value and gets executed every time the attribute is accessed.</font>

In [11]:
class Customer:
    def __init__(self, name, new_bal):
        self.name = name
        if new_bal < 0:
           raise ValueError("Invalid balance!")
        self._balance = new_bal  

    # Add a decorated balance() method returning _balance        
    @property
    def balance(self):
        return self._balance
     
    # Add a setter balance() method
    @balance.setter
    def balance(self, new_bal):
        # Validate the parameter value
        if new_bal<0:
            raise ValueError("Invalid balance!")
        self._balance = new_bal
        # Print "Setter method is called"
        print("Setter method is called")

    # Add a getter balance() method
    @balance.getter
    def balance(self):
        print("Getter method is called")
        return self._balance
    
# Create a Customer        
cust = Customer('Belinda Lutz', 2000)

# Assign 3000 to the balance property
cust.balance = 3000

# Print the balance property
print(cust.balance)

Setter method is called
Getter method is called
3000


In [12]:
%%expect_exception ValueError
# Assign 3000 to the balance property
cust.balance = -3000

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-12-cc2e8909d60e> in <module>
      1 # Assign 3000 to the balance property
----> 2 cust.balance = -3000

<ipython-input-11-1432397095da> in balance(self, new_bal)
     16         # Validate the parameter value
     17         if new_bal<0:
---> 18             raise ValueError("Invalid balance!")
     19         self._balance = new_bal
     20         # Print "Setter method is called"

ValueError: Invalid balance!


# <font color=darkred>4.10 Read-only properties</font> 

The LoggedDF class from Chapter 2 was an extension of the pandas DataFrame class that had an additional created_at attribute that stored the timestamp when the DataFrame was created, so that the user could see how out-of-date the data is.

But that class wasn't very useful: we could just assign any value to created_at after the DataFrame was created, thus defeating the whole point of the attribute! Now, using properties, we can make the attribute read-only.

The LoggedDF class from Chapter 2 is available for you in the script pane.

**Instructions**
- Assign a new value of '2035-07-13' to the created_at attribute.
- Print the value of ldf's created_at attribute to verify that your assignment was successful.


- Create an internal attribute called _created_at to turn created_at into a read-only attribute.
- Modify the class to use the internal attribute, _created_at, in place of created_at.


- What happens when you assign '2035-07-13' to ldf.created_at?
    **Possible Answers**
    - The created_at attribute of ldf is updated to '2035-07-13'.
    - An AttributeError is thrown since '2035-07-13' is not a valid date.
    - An AttributeError is thrown since the created_at attribute doesn't exist.
    - An AttributeError is thrown since ldf.created_at is read-only.

**Results**

<font color=darkgreen>You've put it all together! Notice that the to_csv() method in the original class was using the original created_at attribute. After converting the attribute into a property, you could replace the call to self.created_at with the call to the internal attribute that's attached to the property, or you could keep it as self.created_at, in which case you'll now be accessing the property. Either way works!</font>

In [13]:
import pandas as pd
from datetime import datetime

# LoggedDF class definition from Chapter 2
class LoggedDF(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        pd.DataFrame.__init__(self, *args, **kwargs)
        self.created_at = datetime.today()

    def to_csv(self, *args, **kwargs):
        temp = self.copy()
        temp["created_at"] = self.created_at
        pd.DataFrame.to_csv(temp, *args, **kwargs)   

# Instantiate a LoggedDF called ldf
ldf = LoggedDF({"col1": [1,2], "col2":[3,4]}) 
print(ldf.created_at)

# Assign a new value to ldf's created_at attribute and print
ldf.created_at = '2035-07-13'
print(ldf.created_at)

2021-07-17 23:46:13.452739
2035-07-13


In [14]:
%%expect_exception AttributeError
import pandas as pd
from datetime import datetime

# MODIFY the class to use _created_at instead of created_at
class LoggedDF(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        pd.DataFrame.__init__(self, *args, **kwargs)
        self._created_at = datetime.today()
    
    def to_csv(self, *args, **kwargs):
        temp = self.copy()
        temp["created_at"] = self._created_at
        pd.DataFrame.to_csv(temp, *args, **kwargs)   
    
    # Add a read-only property: _created_at
    @property  
    def created_at(self):
        return self._created_at

# Instantiate a LoggedDF called ldf
ldf = LoggedDF({"col1": [1,2], "col2":[3,4]}) 
ldf.created_at
ldf.created_at = '2035-07-13'

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
C:\Anaconda3\envs\datascience\lib\site-packages\pandas\core\generic.py in __setattr__(self, name, value)
   5495                 else:
-> 5496                     object.__setattr__(self, name, value)
   5497             except (AttributeError, TypeError):

AttributeError: can't set attribute

During handling of the above exception, another exception occurred:

AttributeError                            Traceback (most recent call last)
<ipython-input-14-43d13cab109d> in <module>
     21 ldf = LoggedDF({"col1": [1,2], "col2":[3,4]})
     22 ldf.created_at
---> 23 ldf.created_at = '2035-07-13'

C:\Anaconda3\envs\datascience\lib\site-packages\pandas\core\generic.py in __setattr__(self, name, value)
   5504                         stacklevel=2,
   5505                     )
-> 5506                 object.__setattr__(self, name, value)
   55

# <font color=darkred>4.11 Congratulations!</font>

1. Congratulations!
>Congratulations on completing this course on introduction to object-oriented programming in Python. You've done a great job!

2. Overview
>You learned how to think about your code in terms of classes and objects; how to create attributes and methods. You explored inheritance and polymorphism -- two ideas that allows you to leverage and customize existing code in powerful ways. You also learned the distinction between class-level data and instance-level data. What does it mean for two objects to be equal? Turns out, it can mean anything you want, as you learned in chapter 3. You defined custom equality functions, readable string representations, even built your own exceptions. Finally, you learned what makes a relationship between classes suitable for inheritance, how Python handles private vs public data, and how to use properties to manage data access.

3. What's next?
>So, where can you go from here? You could start by expanding your knowledge of functionality available in Python. For example, learn about mix-in classes and multiple inheritance -- a highly debated feature of Python that isn't present in many other object-oriented languages. You could learn how to override more built-in operators, like arithmetic operators, or the length operator; how to customize attribute access even more using special methods; how to create your own iterator classes that you could use to index loops. You could learn about abstract base classes used to create interfaces, or how leverage dataclasses -- a new type of class that is especially suitable for data storage.

4. What's next?
>Also consider learning more about object-oriented design, which is based on SOLID principles. Solid is an acronym, and you've already learned about the "L" is SOLID -- the Liskov substitution principle, but the other 4 letters are just as important. Finally, I encourage you to learn more about design patterns -- reusable solutions addressing most common problems in software design.

5. Thank you!
>Thank you so much for taking this course, and good luck in you future coding adventures!

# Aditional material

- Datacamp course: https://learn.datacamp.com/courses/object-oriented-programming-in-python
- Circle–ellipse problem: https://en.wikipedia.org/wiki/Circle%E2%80%93ellipse_problem
- Getter and Setter in Python: https://www.geeksforgeeks.org/getter-and-setter-in-python/